In [260]:
using Pkg;
Pkg.activate("..");
Pkg.instantiate();

  Activating project at `~/dev/uni/amo-team-project`


In [261]:
# Pkg.add("Gurobi");
# Pkg.build("Gurobi");
import Gurobi
GRB_ENV = Gurobi.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-19


Gurobi.Env(Ptr{Nothing} @0x000055625e97e210, false, 0)

In [262]:
# using HiGHS
# using JuMP
# using Ipopt
# using DataFrames
# using JSON
# using Plots
# using StatsPlots
# using StatsBase
# using BenchmarkTools
# using Distributions
# using Distances

In [263]:
include("../src/rwth_parser.jl")
network = read_network("../data/Scenario_2013.xlsx")
loads = read_loads("../data/Scenario_2013.xlsx")
print()

In [264]:
include("../src/loads.jl")
local_loads = remove_non_data_rows(filter_month(loads, 1))
print()

In [265]:
include("../src/scenarios/generation.jl")
attack_scenarios = generate_attack_scenarios(network, 10, 1000)
weather_scenarios = generate_weather_scenarios(1000)
print()

In [273]:
include("../src/scenarios/reduction.jl")
reduced_attack_scenarios = reduce_binary_scenarios(attack_scenarios, 5)
reduced_weather_scenarios = reduce_continous_scenarios(weather_scenarios, 5)
reduced_load_scenarios = reduce_continous_scenarios(local_loads, 5)
print()

In [274]:
include("../src/scenarios/assemble.jl")
scenarios = cartesian_scenarios(
    reduced_load_scenarios,
    reduced_attack_scenarios,
    reduced_weather_scenarios,
)
print()

In [275]:
scenarios = translate_weather_to_capacity(scenarios, network.generators)
print()

In [276]:
scenario_dict = convert_df_to_scenarios(scenarios)
print("Number of scenarios: $(length(scenario_dict))")

Number of scenarios: 125

In [277]:
include("../src/model.jl")
dispatch_model = DispatchModel(network, scenario_dict, 5.0)
init_model!(dispatch_model)
print()

In [278]:
t = @time solve!(dispatch_model)
print()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 201501 rows, 94995 columns and 456620 nonzeros
Model fingerprint: 0x3c83952f
Variable types: 79875 continuous, 15120 integer (15120 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [4e-04, 3e-02]
  Bounds range     [6e+01, 7e+02]
  RHS range        [1e+00, 7e+02]
Presolve removed 165635 rows and 48420 columns
Presolve time: 0.39s
Presolved: 35866 rows, 46575 columns, 138275 nonzeros
Variable types: 46535 continuous, 40 integer (40 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   41581    7.4537962e+03   3.082390e+02   1.932845e+08      5s
   47265    5.4087129e+03   1.082188e

In [272]:
import JuMP
print("Objective value: $(JuMP.objective_value(dispatch_model.m))")

Objective value: 1744.5623961710457